# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from nlppen.spacy_internationals import extractInternational
from nlppen.spacy_derechos import extractDerechos


## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "6g")
         .config("spark.executor.cores", "1")
         .config("spark.driver.memory", "12g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/11/02 04:02:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

In [3]:
#df = spark.read.parquet('./datasetRepartido')
#writer = df.write.partitionBy("anno").mode('Overwrite').parquet('./datasetRepartido/split')

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [4]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructuralesNormal')
#seleccion = Seleccion(terminos, spark, parquet_path='./datasetRepartido', datasets_path='./datasets/estructurales')
print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
estructurales = SentenciasEstructurales(seleccion)

Cantidad elementos originales : 1365
Cantidad elementos despues de filtrados : 1365


# Formar dataset de sentencias estructurales

In [5]:
from pyspark.sql.types import *
columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)

columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

columnas = {
    'inst internacionales' : ArrayType(StringType())
}

estructurales.extraerInstrumentosInternacionales(columnas, True)

columnas = {
    'derechos Norm' : ArrayType(StringType()),
    'derechos GenXPat' : ArrayType(StringType())
}
estructurales.extraerDerechos(columnas, True)

columnas = {
    'derechos Acotados' : ArrayType(StringType()),
    'derechos General' : ArrayType(StringType()),
    'derechos Fundamental' : ArrayType(StringType()),
    'derechos Humano' : ArrayType(StringType())
}

estructurales.extraerDerechosSinNormalizar(columnas, True)
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()

21/11/02 04:02:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[archivo: string, txt: string, tema: string, tema_prob: double, expediente: string, fechahora_ext: timestamp, redactor_ext: string, tipoResolucion_ext: string, termino_ext: string, voto_salvado: boolean, Fechahora_ents: array<string>, Redactor_ents: array<string>, CitaSentencias_ents: array<string>, Reglamento_ents: array<string>, termino_lst: array<string>, Cantón_ents: array<string>, Ley_ents: array<string>, EntidadPublica_ents: array<string>, Constitución_ents: array<bigint>, Fecha_ents: array<string>, anno: int, seguimiento: int, se_ordena: int, plan: int, plazo: int, derechos_Norm: array<string>, derechos_GenXPat: array<string>, derechos_Acotados: array<string>, derechos_General: array<string>, derechos_Fundamental: array<string>, derechos_Humano: array<string>]

In [6]:
s = estructurales.seleccion.sdf.repartition(1)
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
s.limit(10).toPandas()

,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,Fechahora_ents,Redactor_ents,...,Fecha_ents,anno,seguimiento,se_ordena,plan,plazo,derechos_Norm,derechos_GenXPat,derechos_Acotados,derechos_General,derechos_Fundamental,derechos_Humano
0,784F6.html,*050151950007CO* Exp: 05-015195-0007-CO Res. Nº 2006-07804 SALA CONSTITUCION...,PETICION,0.086251,291C05,2006-05-30 17:14:00,Armijo Sancho,Recurso de Amparo,Con lugar parcial,False,None,None,...,None,2006,0,1,0,0,"[Derecho Adquirido, Derecho Fundamental a Aprender, Derecho Fundamental a Ap...","[derecho adquirido, derecho de aprender, derecho de aprender]","[derecho privado, derecho prestacional, derecho de la profesora, derecho de ...","[derecho a la educación el, derecho privado, derecho de enseñar el, derecho ...",None,None
1,72C2E.html,*060010870007CO* Exp: 06-001087-0007-CO Res. Nº 2006-02200 SALA CONSTITUCION...,TRABAJO,0.451749,060010870007CO,2006-02-22 14:44:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,None,None,...,None,2006,0,1,0,1,None,None,None,None,None,None
2,7865A.html,*060032250007CO* Exp: 06-003225-0007-CO Res. Nº 2006-010418 SALA CONSTITUCIO...,PENITENCIARIO,0.335066,060032250007CO,2006-07-19 19:41:00,Cruz Castro,Hábeas Corpus,Con lugar,False,[las diecinueve horas y cuarenta y uno minutos del diecinueve de julio del d...,[Cruz Castro],...,"[28 de abril del 2005, 31 de marzo del 2006, 14 de setiembre del 2005, 31 de...",2006,0,0,0,0,None,None,None,None,None,None
3,79A74.html,*060096310007CO* Exp: 06-009631-0007-CO Res. Nº 2006012365 SALA CONSTITUCION...,PETICION,0.252051,060096310007CO,2006-08-23 16:26:00,Calzada Miranda,Recurso de Amparo,Con lugar,False,[las dieciséis horas y veintiséis minutos del veintitrés de agosto del dos m...,[Calzada Miranda],...,"[7 de agosto del 2006, 9 de mayo del 2006, 9 de mayo del 2006, 14 de agosto ...",2006,0,0,0,0,[Derecho Fundamental de Justicia Pronta y Cumplida],[derecho a obtener justicia pronta y cumplida],"[derecho a obtener justicia pronta, derecho fundamental]","[derecho a obtener justicia pronta y cumplida, derecho fundamental]",None,None
4,72D53.html,*050165730007CO* Exp: 05-016573-0007-CO Res: 2006-03283 SALA CONSTITUCIONAL ...,TRABAJO,0.445111,050165730007CO,2006-03-10 13:10:00,Cruz Castro,Recurso de Amparo,Con lugar,False,None,[Cruz Castro],...,"[23 de diciembre del 2005, 17 de noviembre del 2005, 29 de setiembre del 200...",2006,0,0,0,0,[Derecho Fundamental de Justicia Pronta y Cumplida],[derecho a la justicia administrativa],"[derecho estricto, derecho a la justicia administrativa]","[derecho estricto a la constitucionalización de los plazos, derecho a la jus...",None,None
5,7B98B.html,*060035870007CO* Exp: 06-003587-0007-CO Res. Nº 2006010069 SALA CONSTITUCION...,PETICION,0.086251,060035870007CO,NaT,Sosto López,Recurso de Amparo,Con lugar,False,None,[Sosto López],...,"[28 de marzo del 2006, 2 de mayo del 2004, 24 de abril del 2006, 22 de julio...",2006,0,1,0,1,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Vida, Derecho Fund...","[derecho a la salud, derecho a la vida, derecho a la vida, derecho a la inte...","[derecho a la vida protegido, derecho a la salud, derecho de contenido negat...","[derecho fundamental a, derecho a la integridad moral y, derecho a la salud,...",None,None
6,7DA84.html,*060142170007CO* Exp: 06-014217-0007-CO Res. Nº 2006-017920 SALA CONSTITUCIO...,TRABAJO,0.644294,060142170007CO,2006-12-12 17:22:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,[las diecisiete horas y veintidós minutos del doce de Diciembre del dos mil ...,[Jinesta Lobo],...,"[20 de noviembre del 2006, 11 de octubre de 2005, 19 de enero del 2006, 5 de...",2006,0,1,0,1,"[Derecho Fundamental a un Procedimiento Administrativo, Derecho Fundamental ...","[derecho a un procedimiento administrativo, DERECHO A UN PROCEDIMIENTO ADMIN...","[derecho a una justicia pronta, derecho a un procedimiento administrativo]","[derecho a una justicia pronta y cumplida d

In [7]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


/tmp/ipykernel_17634/3051205347.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [8]:

df2 = (s
 .where(s.derechos_Norm.isNotNull())
 .select(explode('derechos_Norm').alias('derechos_Norm'), 'anno')
 .groupby('derechos_Norm')
 .count()
 .sort(F.desc('count'))
).toPandas()

file_name = 'derechos_Norm.xlsx'
  
# saving the excel
df2.to_excel(file_name)

In [9]:

df2 = (s
 .where(s.derechos_GenXPat.isNotNull())
 .select(explode('derechos_GenXPat').alias('derechos_GenXPat'), 'anno')
 .groupby('derechos_GenXPat')
 .count()
 .sort(F.desc('count'))
).toPandas()

file_name = 'derechos_GenXPat.xlsx'
  
# saving the excel
df2.to_excel(file_name)

In [10]:
df2 = (s
 .where(s.derechos_Acotados.isNotNull())
 .select(explode('derechos_Acotados').alias('derechos_Acotados'), 'anno')
 .groupby('derechos_Acotados')
 .count()
 .sort(F.desc('count'))
).toPandas()

file_name = 'derechos_Acotados.xlsx'
  
# saving the excel
df2.to_excel(file_name)


In [17]:
df2 = (s
 .where(s.derechos_fundamental.isNotNull())
 .select(explode('derechos_fundamental').alias('derechos_fundamental'), 'anno')
 .groupby('derechos_fundamental')
 .count()
 .sort(F.desc('count'))
).toPandas()

file_name = 'Derechos fundamental.xlsx'
  
# saving the excel
df2.to_excel(file_name)

In [18]:
df2 = (s
 .where(s.derechos_humano.isNotNull())
 .select(explode('derechos_humano').alias('derechos_humano'), 'anno')
 .groupby('derechos_humano')
 .count()
 .sort(F.desc('count'))
).toPandas()

file_name = 'Derechos humano.xlsx'
  
# saving the excel
df2.to_excel(file_name)

In [ ]:
df2 = (s
 .where(s.derechos_General.isNotNull())
 .select(explode('derechos_General').alias('derechos_General'), 'anno')
 .groupby('derechos_General')
 .count()
 .sort(F.desc('count'))
).toPandas()

file_name = 'derechos_General.xlsx'
  
# saving the excel
df2.to_excel(file_name)

# Cargar procesamiento de sentencias estructurales.

In [12]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
s = estructurales.seleccion.sdf
s.limit(10).toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,se_ordena_Ent_Pub,extension_sentencia,extension_por_lo_tanto,plazosDefinidos,FechaSolicitud,num_resolucion
0,784F6.html,*050151950007CO* Exp: 05-015195-0007-CO Res. N...,PETICION,0.086251,291C05,2006-05-30 17:14:00,Armijo Sancho,Recurso de Amparo,Con lugar parcial,False,...,"[Circuito 02 de la, Colegio Líder INVU Las Cañas]","[Directora de la Escuela, Dirección Regional d...","[Director Regional de Educación de Alajuela, I]",None,"[Instituto Nacional de Vivienda y Urbanismo, I...",24112,2582,None,2005-11-24 10:05:00,2006-07804
1,72C2E.html,*060010870007CO* Exp: 06-001087-0007-CO Res. N...,TRABAJO,0.451749,060010870007CO,2006-02-22 14:44:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,"[Ministerio de Educación Pública, Ministro de ...",None,None,None,[Ministerio de Educacion Publica],6022,1394,[1970-01-03 00:00:00],2006-01-31 12:27:00,2006-02200
2,7865A.html,*060032250007CO* Exp: 06-003225-0007-CO Res. N...,PENITENCIARIO,0.335066,060032250007CO,2006-07-19 19:41:00,Cruz Castro,Hábeas Corpus,Con lugar,False,...,None,None,None,None,None,4704,1297,None,2005-04-28 14:13:00,2006-010418
3,79A74.html,*060096310007CO* Exp: 06-009631-0007-CO Res. N...,PETICION,0.252051,060096310007CO,2006-08-23 16:26:00,Calzada Miranda,Recurso de Amparo,Con lugar,False,...,None,None,None,None,None,6041,457,None,2006-08-07 14:05:00,2006012365
4,72D53.html,*050165730007CO* Exp: 05-016573-0007-CO Res: 2...,TRABAJO,0.445111,050165730007CO,2006-03-10 13:10:00,Cruz Castro,Recurso de Amparo,Con lugar,False,...,None,None,None,None,None,4824,403,None,2005-12-23 14:20:00,2006-03283
5,7B98B.html,*060035870007CO* Exp: 06-003587-0007-CO Res. N...,PETICION,0.086251,060035870007CO,NaT,Sosto López,Recurso de Amparo,Con lugar,False,...,"[Ministra de Obras Publicas y Transportes, Min...",None,[Dependencias Técnicas],None,None,19759,1575,[1970-04-01 00:00:00],2006-03-28 09:41:00,2006010069
6,7DA84.html,*060142170007CO* Exp: 06-014217-0007-CO Res. N...,TRABAJO,0.644294,060142170007CO,2006-12-12 17:22:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,...,[Ministerio de Trabajo y Seguridad],[Director Nacional de Pensiones],None,None,None,14270,1466,None,2006-11-20 11:00:00,2006-017920
7,7544C.html,*060045980007CO* Exp: 06-004598-0007-CO Res. N...,TRABAJO,0.371132,060045980007CO,2006-05-16 15:54:00,Armijo Sancho,Recurso de Amparo,Con lugar,False,...,None,None,None,None,None,6579,401,None,2006-04-24 13:06:00,2006-006683
8,715B6.html,*050147230007CO* Exp: 05-014723-0007-CO Res. N...,PETICION,0.273704,050147230007CO,2006-02-10 11:51:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,[Alcaldesa Municipal de Montes de Oca],None,None,None,None,9301,1275,None,2005-11-15 08:52:00,2006-001463
9,6FF2A.html,*050143170007CO* Exp: 05-014317-0007-CO Res. N...,PENSION,0.680464,050143170007CO,2006-01-27 12:25:00,Molina Quesada,Recurso de Amparo,Con lugar,False,...,None,[Director Nacional de Pensiones],"[DNP-RE-2888-05, Ley de la Jurisdicción Consti...",None,None,6599,1453,[1970-06-30 00:00:00],2005-11-07 08:10:00,2006-00694


# Zona de trabajo (ignorar)

In [6]:
s = estructurales.seleccion.sdf
print("Totales :", str(s.count()))
print("Nulos : ", str(s.filter(s.derechos.isNull()).count()))
print("Completos : ", str(s.filter(s.derechos.isNotNull()).count()))
print("Accuracy : " ,s.filter(s.derechos.isNotNull()).count()/s.count())

#s = s.filter(s.num_resolucion.isNull())
#s = estructurales.seleccion.sdf

Totales : 1365


AttributeError: 'DataFrame' object has no attribute 'derechos'

In [9]:
s.filter(s.inst_internacionales.isNull()).groupby(s.anno).count().toPandas().sort_values('anno')

,anno,count
0,1990,3
12,1991,6
20,1992,7
29,1993,8
8,1994,12
18,1995,5
13,1996,18
7,1997,29
14,1998,28
27,1999,17


In [12]:
s.filter(s.inst_internacionales.isNotNull()).groupby(s.anno).count().toPandas().sort_values('anno')

,anno,count
6,1997,2
10,1998,1
15,2000,1
0,2003,1
9,2004,2
14,2005,4
4,2006,5
1,2007,16
18,2008,9
12,2009,8


In [7]:
from nlppen.spacy_internationals import extractInternational
from nlppen.spark_udfs import *
doc = extractInternational("a veces tenemos Protocolo a la Convención Americana sobre Derechos Humanos en materia de Derechos Económicos, Sociales y Culturales")
entidadesFiltradas = [ent.ent_id_ for ent in doc.ents]
print(entidadesFiltradas)

['Protocolo a la Convención Americana sobre Derechos Humanos en materia de Derechos Económicos, Sociales y Culturales']


In [24]:
s.filter(s.inst_internacionales.isNotNull()).limit(10).toPandas().iloc[8,-1]

array(['Convención sobre los Derechos de las Personas con Discapacidad y su Protocolo'],
      dtype=object)

In [ ]:
s.limit(10).toPandas()

In [3]:

from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy

In [4]:
nlp = spark_get_spacy("es_core_news_lg")

In [5]:
txt = """
Derecho a tener posesiones
"""
doc = nlp(txt)
words = []
for word in doc:
    words.append(word.text)
    print(word.text, word.pos_, word.is_stop)
print("----")
union = list(set().union(words, words))
for word in union:
    print(word)


 SPACE False
derecho NOUN False
fundamental ADJ False
de ADP True
todo DET True
imputado ADJ False
de ADP True
solicitar VERB False
o CCONJ False
pedir VERB False
al ADP True
Juez PROPN False
Penal PROPN False

 SPACE False
----
de
al
Juez
derecho
todo


Penal
imputado
pedir
o
fundamental
solicitar


In [1]:

from nlppen.spacy_derechos import extractDerechos
txt = """
Derecho a tener posesiones

"""
doc = extractDerechos(txt)
entidadesFiltradas = [ent.ent_id_ for ent in doc.ents]
entidadesSacadas = [ent.text for ent in doc.ents]
print(entidadesFiltradas)
print(entidadesSacadas)

['Derecho Fundamental a tener Posesiones']
['Derecho a tener posesiones']


In [38]:
import re 
derechos = ["derecho   a la salud", "derecho a la salud  ", "derecho a la salud "]
derechos = [re.sub(r'[^\w]', ' ', x) for x in derechos]
derechos = [re.sub(r'(\s)+', ' ', x.strip()) for x in derechos]
union = list(set().union(derechos, derechos))
print(union)

['derecho a la salud']


Derecho   a laa   salud


In [36]:
ss = s.filter(s.inst_internacionales.isNull()).filter(s.anno == "1990")

In [14]:
from spacy.matcher import Matcher
from spacy.tokens import Span

import re

def limpiarDerechos(derechos):
    union = list(set().union(derechos, derechos))
    return union
    
def crearDerechos(txt):
    derechos = []
    se_ordena_pattern = [
                         {"LOWER": "derecho"}, {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "+"},
                         {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "*"},
                         {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "*"}
                        ]
    fundamental = [
                         {"LOWER": "derecho"}, {"LOWER": "fundamental"}, {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "+"},
                         {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "*"},
                         {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "*"}
                        ]
    humano = [
                         {"LOWER": "derecho"}, {"LOWER": "humano"}, {"POS": {"IN":["PRON", "VERB", "DET"]}, "OP": "*"}, {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "+"},
                         {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "*"},
                         {"POS": {"IN":["ADP", "DET"]}, "OP": "*"},
                         {"LOWER": {"IN": ["y", "o"]}, "OP": "?"},
                         {"POS": {"IN": ["VERB", "ADJ", "NOUN"]}, "OP": "*"}
                        ]

    patterns = [se_ordena_pattern]
    nlp = spark_get_spacy("es_core_news_lg")
    doc = nlp(txt)
    matcher = Matcher(nlp.vocab)
    matcher.add("Derecho General", patterns, greedy="FIRST")
    matcher.add("Derecho Fundamental", [fundamental], greedy="FIRST")
    matcher.add("Derecho Humano", [humano], greedy="FIRST")
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = Span(doc, start, end, label=match_id)
        if span.label_ == "Derecho Humano":
            print(span.label_)
            print(span.text)
        derechos.append(span.text)
        print("*-*-*-*")
        
    #for derecho in derechos:
    #    print(derecho)
    
    derechos = limpiarDerechos(derechos)
    return derechos

#v = s.count()
v = 150
pand = s.limit(v).toPandas()
derechosExtraidos = []
for i in range(50, v):
    print("************************")
    print(pand.iloc[i, 4])
    derechosExtraidos += crearDerechos(solo_considerando(pand.iloc[i,1]).lower())

derechosExtraidos = [re.sub(r'[^\w]', ' ', x) for x in derechosExtraidos]
derechosExtraidos = limpiarDerechos(derechosExtraidos)
#for derecho in derechosExtraidos:
#    print(derecho)

************************
180034830007CO
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
************************
9616A00
*-*-*-*
*-*-*-*
*-*-*-*
************************
180047710007CO
*-*-*-*
************************
180063250007CO
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
************************
180084130007CO
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
Derecho Humano
derecho humano que tiene el derecho a la salud de todos los ciudadanos
*-*-*-*
************************
180010110007CO
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
*-*-*-*
Derecho Humano
derecho humano para la población adulta mayor y el precepto
*-*-*-*
*-*-*-*
************************
180008910007CO
*-*-*-*
*-*-*-*
************************
180004010007CO
*-*-*-*
*-*-*-*
************************
180053820007CO
*-*-*-*
************************
180040570007CO
*-*-*-*
*-*-*-*
*-*-*-*
**********************

In [41]:
a = ["Abc", "Abc", "CCC"]
union=list(set().union(a, a))
print("\nThe union of three list is:",union)


The union of three list is: ['CCC', 'Abc']


In [15]:
a = [1, 2, 3]

In [16]:
b = [4, 5, 6]

c = [a, b]

In [17]:
print(c)

[[1, 2, 3], [4, 5, 6]]
